### Q. Any idea on managing these 200+ features after ohe?

In [ ]:
# ## for colab
# # !pip install -U -q PyDrive

# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
# ## for colab
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

In [ ]:
## common imports
import numpy as np
import pandas as pd
import os
import pickle
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

In [ ]:
## to plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

## to make this notebook's output stable across runs
np.random.seed(42)

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
## import sklearn packages
# from sklearn.metrics import precision_score, recall_score
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
# from sklearn.metrics import roc_curve, auc, roc_auc_score
# from sklearn.metrics import precision_recall_curve

# 01_ReadData

In [ ]:
## for colab 
## read the data
# df_all = pd.read_csv('/content/drive/My Drive/Final Project_Nisal David Elsa/insurance_classification_data.csv')  

In [ ]:
## read in the raw data to a data frame and saves it to pickle
# df_all = pd.read_csv('../final_project_data/insurance_classification_data.csv')
# with open('../final_project_data/insurance_clf_data.pickle','wb') as file:
#     pickle.dump(df_all, file)

In [ ]:
## read the data
with open('../final_project_data/insurance_clf_data.pickle','rb') as file:
    df_all = pickle.load(file)

In [ ]:
df_all.shape

In [98]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1794135 entries, 0 to 1795671
Data columns (total 89 columns):
uniqueID    int64
Var01       object
Var03       object
Var07       int64
Var10       float64
Var11       object
Var13       object
Var14       object
Var19       object
Var20       object
Var21       object
Var23       object
Var25       object
Var26       float64
Var27       float64
Var28       float64
Var29       float64
Var30       int64
Var31       int64
Var32       int64
Var33       int64
Var34       int64
Var35       int64
Var42       int64
pred_s      float64
Var47       float64
Var48       object
Var49       object
Var50       int64
Var51       object
Var52       object
Var53       object
Var55       object
c01         object
c02         object
c03         object
c06         object
c07         object
c08         float64
c09         float64
c10         float64
c11         float64
c12         float64
c13         object
c14         object
c15         float64
c17       

In [ ]:
df_all.head()

# 02a_DataManipulation_basic

#### Remove duplicates

In [ ]:
## check duplicates
duplicateRowsDF = df_all[df_all.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
duplicateRowsDF.shape

In [ ]:
df_all = df_all.drop_duplicates()

In [ ]:
print(df_all.shape)

# 02b_DataManipulations_missings

#### replacing missing by the mode if it is categorical and the mean if numeric

In [66]:
## check missing values
def check_missings(df):
    total = df.isnull().sum()
    percent = df.isnull().sum()/df.isnull().count()*100
    missing = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing

In [67]:
missing_list = check_missings(df_all)

In [70]:
print(missing_list[missing_list.Total>0].sort_values(by='Total',ascending=False))

         Total    Percent
c04     728781  40.620187
c39     444988  24.802370
c40     439965  24.522402
c38     427914  23.850714
c16     260952  14.544725
c05     253812  14.146762
id      192542  10.731745
c18     172688   9.625140
c15     170015   9.476154
c08     169206   9.431063
c09     169206   9.431063
c10     169206   9.431063
pred_s  147947   8.246146
Var47   147865   8.241576
c19      78619   4.382000
c26      60661   3.381072
c27      42452   2.366154
c34      28487   1.587785
c35      28487   1.587785
c12      12204   0.680216
c11      10673   0.594883
Var12     6587   0.367141
Var02       13   0.000725
Var10       13   0.000725
Var29        5   0.000279
Var28        5   0.000279
Var26        4   0.000223
Var27        4   0.000223


In [81]:
indicator_cols = missing_list.loc[missing_list['Total']>0].drop_duplicates().index.values.tolist()

In [82]:
indicator_cols

['Var10',
 'Var26',
 'Var28',
 'pred_s',
 'Var47',
 'c08',
 'c11',
 'c12',
 'c15',
 'c18',
 'c19',
 'c27',
 'Var12',
 'c04',
 'c05',
 'c16',
 'c26',
 'c34',
 'c38',
 'c39',
 'c40',
 'id']

<p>** A little confused with the following instruction. **<br> So you could create one missing indicator variable and use it for all of them.  Then for example if you create a logistic model and the target follows a different  pattern for examples with imputed mean than for cases that really are at the mean  (which is almost certainly will) then the missing indicator can pick that up.

In [ ]:
print(list(df_all))

In [85]:
target = ['Var47']

labels = ['uniqueID','id', 'train', 'Var47']

features = ['Var01', 'Var03', 'Var07', 'Var10', 'Var11', 'Var13',
             'Var14', 'Var19', 'Var20', 'Var21', 'Var23', 'Var25', 'Var26',
             'Var27', 'Var28', 'Var29', 'Var30', 'Var31', 'Var32', 'Var33', 
             'Var34', 'Var35', 'Var42', 'Var48', 'Var49', 
             'Var50', 'Var51', 'Var52', 'Var53', 'Var55', 
             'c01', 'c02', 'c03', 'c06', 'c07', 'c08', 'c09', 'c10', 'c11', 'c12', 
             'c13', 'c14', 'c15', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 
             'c24', 'c25', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c36', 'c37', 
             'Var54', 'Var56', 'Var02', 'Var04', 'Var05', 'Var06', 'Var08', 'Var09', 
             'Var12', 'Var15', 'Var16', 'Var18', 'Var22', 'Var24', 
             'c04', 'c05', 'c16', 'c26', 'c34', 'c35', 'c38', 'c39', 'c40']


na_indicator_cols = ['Var10', 'Var26', 'Var28', 'Var47', 'c08','c11', 'c12', 'c15',
                          'c18', 'c19', 'c27', 'Var12', 'c04', 'c05', 'c16', 'c26', 
                          'c34', 'c38', 'c39', 'c40',]

In [86]:
df_na_indicators = df_all[na_indicator_cols].isnull().astype(int).add_suffix('_NA')

In [87]:
## Remove duplicated Missing Indicators

# df_na_indicators = df_all[features].isnull().astype(int).add_suffix('_NA')

# def duplicate_columns(df):
#     """Find duplicate columns in a DataFrame based on their values.
    
#     [Source] https://github.com/pandas-dev/pandas/issues/11250 
    
#     """   
#     groups = df.columns.to_series().groupby(df.dtypes).groups
#     dups = []

#     for t, v in groups.items():

#         cs = df[v].columns
#         vs = df[v]
#         lcs = len(cs)

#         for i in range(lcs):
#             iv = vs.iloc[:,i].tolist()
#             for j in range(i+1, lcs):
#                 jv = vs.iloc[:,j].tolist()
#                 if iv == jv:
#                     dups.append(cs[i])
#                     break

#     return dups


# dup_col_list = duplicate_columns(df_na_indicators)
# df_na_indicators = df_na_indicators.drop(duplicate_columns_list, axis=1)
# print(df_na_indicators.columns)

In [88]:
## Impute missing values 
## [Source] https://stackoverflow.com/questions/25239958/impute-categorical-missing-values-in-scikit-learn

from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [89]:
df_all_filled = DataFrameImputer().fit_transform(df_all)
# print(df_all_filled.head().transpose())

df_all_a = pd.concat([df_all_filled, df_na_indicators], axis=1) 

df_all_b = pd.get_dummies(df_all_a)

# with open('../final_project_data/insurance_clf_data_new.pickle','wb') as file:
#     pickle.dump(df_all_b, file)

# 03a_EDA_basic

### check for distributions, correlation, ...
<p>Q. how to deal with categorical data?
<p>Todo. correlcation analysis

#### Train test split

In [102]:
## training set
df_train = df_all.loc[df_all.train=="Y"]
print(df_train.shape)

(1646270, 89)


In [ ]:
## test set
df_test = df_all.loc[df_all.train=="N"]
print(df_test.shape)

In [ ]:
# # Split numeric and categorical features for training set
# cat_train_feats = data_train.select_dtypes(include=['object']).columns
# num_train_feats = data_train.select_dtypes(include=['float64','int64']).columns
# num_train_df = data_train[num_train_feats]
# cat_train_df = data_train[cat_train_feats]
# print('There are %d numeric features and %d categorical features in the training dataset\n' %(len(num_train_feats),len(cat_train_feats)))
# print('Numeric features:\n',num_train_feats.values)
# print('Categorical features:\n',cat_train_feats.values)

#### Unique values within each variable

In [ ]:
# Look at some of the unique values within each variable | traning set
for col in list(data_train.columns):
    uniques = data_train[col].unique() # get a list of unique values  
    uniques.sort()
    # if number of unique values is less than 30, print the values. Otherwise print the number of unique values
    if len(uniques)<30:
        print(col + ':')
        print(uniques)
    else:
        print(col + ': ' + str(len(uniques)) + ' unique values')

# 03a_EDA_distribution

In [103]:
def cat_barplot(var):
    clean_var = df_train[var].fillna('Missing')
    var_counts = clean_var.value_counts()
    barplot = sns.barplot(x=var_counts.index, y=var_counts.values, alpha=1)
    return var_counts, barplot

In [ ]:
# plt.figure(figsize=(16,16))
# plt.subplot(221); cat_barplot('Var03')
# plt.subplot(222); cat_barplot('Var01')

In [ ]:
cat_barplot('c04')

In [ ]:
plt.figure(figsize=(16,16))
cat_barplot('c05')

# 03b_EDA_boxplot

#### Categorical data & odrinary data, relation with classification target 0/1 (Boxplots)

In [ ]:
df_train.boxplot('c04','Var47', rot = 30,figsize=(5,6))

# 03a_EDA_pred_s

#### Exsiting linear model accuracy pred_s  |  threshold = 50
<p> the following could be replaced by confusion matrix

In [ ]:
Var47_counts = data_train['Var47'].value_counts()
print(Var47_counts)

In [ ]:
TP = len(data_train[(data_train['pred_s']>=50) & (data_train['Var47']==1)]) 
FN = len(data_train[(data_train['pred_s']<50) & (data_train['Var47']==0)]) 
accuracy = (TP+FN)/(data_train.shape[0])
print(accuracy)

# 04a_LinearModel

<p>Q. How to impute missing values
<p>Q. What could we do with categorical data ABCDE? What could we do with discreate numerical data 1 2 3 4 5?
<p>Q. In theory, we can only use training set for model building, however, the categorical data make onehot encoding complicated if we conduct it for train and test set seperately. And error "X has 254 features per sample; expecting 273" happened. Is there any good way to do it?

In [109]:
## read the data
with open('../final_project_data/insurance_clf_data_new.pickle','rb') as file:
    df_all = pickle.load(file)

In [112]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1794135 entries, 0 to 1795671
Columns: 299 entries, uniqueID to train_Y
dtypes: float64(28), int64(41), uint8(230)
memory usage: 1.3 GB


#### Select input data for training models

In [134]:
## All variables in the transformed dataframe
features = list(df_all)
df_features = pd.DataFrame(np.array(list(df_all)).reshape(23, 13))
df_features

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,uniqueID,Var07,Var10,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35
1,Var42,pred_s,Var47,Var50,c08,c09,c10,c11,c12,c15,c18,c19,c27
2,Var54,Var02,Var04,Var05,Var06,Var08,Var09,Var12,Var15,Var16,Var18,Var22,Var24
3,c04,c05,c16,c26,c34,c35,c38,c39,c40,id,Var10_NA,Var26_NA,Var28_NA
4,Var47_NA,c08_NA,c11_NA,c12_NA,c15_NA,c18_NA,c19_NA,c27_NA,Var12_NA,c04_NA,c05_NA,c16_NA,c26_NA
5,c34_NA,c38_NA,c39_NA,c40_NA,Var01_A,Var01_B,Var01_C,Var03_A,Var03_B,Var03_C,Var03_D,Var03_E,Var03_F
6,Var03_G,Var03_H,Var03_I,Var03_J,Var03_L,Var03_N,Var03_P,Var03_Q,Var03_Z,Var11_A,Var11_B,Var11_C,Var11_D
7,Var11_E,Var11_F,Var11_G,Var11_Z,Var13_A,Var13_B,Var13_C,Var13_D,Var13_E,Var13_F,Var13_G,Var13_H,Var13_I
8,Var13_J,Var13_K,Var13_L,Var13_M,Var13_N,Var13_O,Var13_P,Var13_Q,Var13_R,Var13_S,Var13_Z,Var14_A,Var14_B
9,Var14_C,Var14_D,Var14_E,Var14_F,Var14_Z,Var19_A,Var19_B,Var19_C,Var19_D,Var19_E,Var19_F,Var19_G,Var19_Z


In [135]:
## setting the inputs the output for our classification task
target = ['Var47']

labels = ['uniqueID', 'id', 'train_N', 'train_Y', 'Var47','pred_s']

variables = ['Var01', 'Var03', 'Var07', 'Var10', 'Var11', 'Var13',
             'Var14', 'Var19', 'Var20', 'Var21', 'Var23', 'Var25', 'Var26',
             'Var27', 'Var28', 'Var29', 'Var30', 'Var31', 'Var32', 'Var33', 
             'Var34', 'Var35', 'Var42', 'Var48', 'Var49', 
             'Var50', 'Var51', 'Var52', 'Var53', 'Var55', 
             'c01', 'c02', 'c03', 'c06', 'c07', 'c08', 'c09', 'c10', 'c11', 'c12', 
             'c13', 'c14', 'c15', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 
             'c24', 'c25', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c36', 'c37', 
             'Var54', 'Var56', 'Var02', 'Var04', 'Var05', 'Var06', 'Var08', 'Var09', 
             'Var12', 'Var15', 'Var16', 'Var18', 'Var22', 'Var24', 
             'c04', 'c05', 'c16', 'c26', 'c34', 'c35', 'c38', 'c39', 'c40']

In [139]:
## setting the inputs the output for our classification task
features_01 = list(set(features) - set(labels))
print(len(features_01))

293


#### df_train & df_test

In [147]:
df_train = df_all.loc[df_all.train_Y==1]
df_test = df_all.loc[df_all.train_Y==0]
print(df_train.shape, df_test.shape)

(1646270, 299) (147865, 299)


#### X_train  & y_train

In [150]:
X_train_01 = df_train[features_01].values.astype(np.int)
y_train = df_train[target].values.astype(np.int)
print(X_train_01.shape, y_train.shape)

(1646270, 293) (1646270, 1)


#### X_test & y_test

In [151]:
X_test_01 = df_test[features_01].values.astype(np.int)
y_test = df_test[target].values.astype(np.int)
print(X_test_01.shape, y_test.shape)

(147865, 293) (147865, 1)


#### Sampling (This part needs furture improvement)
<p> Limited by our computing resources, we are now using a small sample of data to train models.

In [ ]:
from sklearn.model_selection import train_test_split
df_train_0_b, df_train_0_s = df_test_split(df_train_0, test_size=0.3, random_state=42)
print(df_train_0_b.shape, df_train_0_s.shape)

In [ ]:
X0_train_s = df_train_0_s.drop(labels, axis=1).fillna(0).values.astype(np.int8)
y0_train_s = df_train_0_s[target].values.astype(np.int8)
print(X0_train_s.shape, y0_train_s.shape)

<p> Considering stratified sampling?

In [ ]:
# # Divide by 1.5 to limit the number of income categories
# housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
# # Label those above 5 as 5
# housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

In [ ]:
# from sklearn.model_selection import StratifiedShuffleSplit

# split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# for train_index, test_index in split.split(housing, housing["income_cat"]):
#     strat_train_set = housing.loc[train_index]
#     strat_test_set = housing.loc[test_index]

In [ ]:
# def income_cat_proportions(data):
#     return data["income_cat"].value_counts() / len(data)

# train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

# compare_props = pd.DataFrame({
#     "Overall": income_cat_proportions(housing),
#     "Stratified": income_cat_proportions(strat_test_set),
#     "Random": income_cat_proportions(test_set),
# }).sort_index()
# compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
# compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [ ]:
# compare_props

In [ ]:
# for set_ in (strat_train_set, strat_test_set):
#     set_.drop("income_cat", axis=1, inplace=True)

#### Performance measurements

In [ ]:
# This is a function which outputs a variety of accuracy results. We can use these results to compare models.

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
def calc_specificity(y_vals, preds, threshes):
    # calculates specificity
    return sum((preds < threshes) & (y_vals == 0)) /sum(y_vals ==0)

def print_report(y_vals, preds, threshes):
    
    auc = roc_auc_score(y_vals, preds)
    accuracy = accuracy_score(y_vals, (preds > threshes))
    recall = recall_score(y_vals, (preds > threshes))
    precision = precision_score(y_vals, (preds > threshes))
    specificity = calc_specificity(y_vals, preds, threshes)
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('specificity:%.3f'%specificity)
    print(' ')
    return auc, accuracy, recall, precision, specificity 

#### Training models and validation
<p> Toooo low validation accuracy now.

In [ ]:
# ## use a small sample; 30% of training data
# X_train = X0_train_s
# y_train = y0_train_s
# X_test = X0_test
# y_test = y0_test

In [152]:
## 
X_train = X_train_01
y_train = y_train
X_test = X_test_01
y_test = y_test

In [153]:
# prepare Logistic Regression classifier
from sklearn.linear_model import LogisticRegression
logit_clf = LogisticRegression(tol=0.0001, C=1.0, random_state=42, max_iter=100,penalty='l1', solver='liblinear',verbose=0)  

In [154]:
# training and validation
import time
start_time = time.time()
logit_clf.fit(X_train, y_train) 
print("--- %s seconds ---" % (time.time() - start_time))

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


--- 710.2725329399109 seconds ---


In [155]:
y_train_logit_pred = logit_clf.predict(X_train)
y_test_logit_pred = logit_clf.predict(X_test)

In [156]:
print("Training Accuracy", accuracy_score(y_train, y_train_logit_pred))
print("Test Accuracy", accuracy_score(y_test, y_test_logit_pred))

Training Accuracy 0.7513706743122331
Test Accuracy 0.5159300713488655


# 05_a. DecisionTree

In [ ]:
## use all training data
X_train = X0_train
y_train = y0_train
X_test = X0_test
y_test = y0_test

In [ ]:
## prepare Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=42, max_depth = 20)

In [ ]:
import time
start_time = time.time()
rf_clf.fit(X_train, y_train) 
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
y_train_rf_pred = rf_clf.predict(X_train)
y_test_rf_pred = rf_clf.predict(X_test)

In [ ]:
print("Training Accuracy", accuracy_score(y_train, y_train_rf_pred))
print("Test Accuracy", accuracy_score(y_test, y_test_rf_pred))

#### Feature importance 

In [ ]:
## Source： https://towardsdatascience.com/feature-selection-techniques-in-machine-learning-with-python-f24e7da3f36e
X = X_train
y = y_train
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

# 06_a.DNN

In [ ]:
X_train = X0_train_s
y_train = y0_train_s
X_test = X0_test
y_test = y0_test

In [ ]:
import tensorflow as tf
from tensorflow import keras
# from tensorflow_graph_in_jupyter import show_graph

In [ ]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [ ]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "."    
logdir = "{}//run-{}//".format(root_logdir, now)

In [ ]:
## define validation and training sets
X_test = X_test.astype(np.int8)
y_train = y_train.astype(np.int8)
y_test = y_test.astype(np.int8)
X_valid, X_train = X_train[:100], X_train[100:]
y_valid, y_train = y_train[:100], y_train[100:]


n_inputs = 273  # number of features 
n_hidden1 = 150   
n_hidden2 = 70   
# dedfine more hidden layers
n_hidden3 = 50
n_outputs = 2 
tf.reset_default_graph()   # reset graph


# y place holder for output
# auxialrry variable 
X = tf.placeholder(tf.int8, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int8, shape=(None), name="y")
a = tf.placeholder(tf.int8, shape=(None),name='a')


summary1 = tf.reduce_mean(a, name="summary1")

#layer of network takes input of neurones  
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    logits  = neuron_layer(hidden3, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

#learning rate - change if issues regrading convergence or others happen
learning_rate =1e-1
with tf.name_scope("train"):
    optimizer = tf.train.AdagradOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

#define accuracy 
with tf.name_scope("accuracy"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

#leave this
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [ ]:
from random import randint
n_epochs = 100  
batch_size = 100   

init = tf.global_variables_initializer()
saver = tf.train.Saver()


summary2 = tf.summary.scalar('Accuracy', summary1)
file_writer = tf.summary.FileWriter(logdir , tf.get_default_graph())

start_time = time.time()
with tf.Session() as sess:
    init.run()
    acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    print("Val accuracy init:", acc_val)
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)
        
        summary_str = summary2.eval(feed_dict={a: acc_val })
        file_writer.add_summary(summary_str, epoch)
   
    saver.save(sess, "./insurance_dnn_model.ckpt")   

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
with tf.Session() as sess:                         
    saver.restore(sess, "./insurance_dnn_model.ckpt")   
    Z = logits.eval(feed_dict={X: X_test})
    y_test_dnn_pred = np.argmax(Z, axis=1)

In [ ]:
print("Training Accuracy", accuracy_score(y_train, y_train_dnn_pred))
print("Test Accuracy", accuracy_score(y_test, y_test_dnn_pred))